# 多文档智能体

在这个notebook中，我们将了解如何使用 `DocumentAgents` 概念和 `ReAct Agent` 来构建处理大量文档的RAG系统。

### 安装

In [ ]:
!pip install llama-index
!pip install llama-index-llms-anthropic
!pip install llama-index-embeddings-huggingface

### 设置日志

In [ ]:
# 注意：这仅在jupyter notebook中是必需的。
# 详情：Jupyter在后台运行事件循环。
#          当我们启动事件循环进行异步查询时，这会导致嵌套的事件循环。
#          这通常是不允许的，我们使用nest_asyncio来允许它以便使用方便。
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

# 设置根日志记录器
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # 设置日志记录器级别为INFO

# 清除任何现有的处理器
logger.handlers = []

# 设置StreamHandler以输出到sys.stdout（Colab的输出）
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)  # 设置处理器级别为INFO

# 将处理器添加到日志记录器
logger.addHandler(handler)

from IPython.display import display, HTML

### 设置Claude API密钥

In [ ]:
os.environ["ANTHROPIC_API_KEY"] = "请输入您的Claude API密钥"

### 设置LLM和嵌入模型

我们将使用anthropic最新发布的 `Claude-3 Opus` LLM。

In [3]:
from llama_index.llms.anthropic import Anthropic
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [4]:
llm = Anthropic(temperature=0.0, model="claude-opus-4-1")
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [5]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

### 下载文档

我们将使用 `Toronto`、`Seattle`、`Chicago`、`Boston`、`Houston` 这些城市的维基百科页面来构建RAG流水线。

In [6]:
wiki_titles = ["Toronto", "Seattle", "Chicago", "Boston", "Houston"]

from pathlib import Path

import requests

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            # 'exintro': True,
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

### 加载文档

In [ ]:
# 加载所有维基百科文档

from llama_index.core import SimpleDirectoryReader

city_docs = {}
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()

#### 为每个城市构建ReAct智能体

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# 构建智能体字典
agents = {}

for wiki_title in wiki_titles:
    # 构建向量索引
    vector_index = VectorStoreIndex.from_documents(
        city_docs[wiki_title],
    )
    # 构建摘要索引
    summary_index = SummaryIndex.from_documents(
        city_docs[wiki_title],
    )
    # 定义查询引擎
    vector_query_engine = vector_index.as_query_engine()
    summary_query_engine = summary_index.as_query_engine()

    # 定义工具
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=(f"用于从{wiki_title}检索特定上下文"),
            ),
        ),
        QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=(f"用于处理与{wiki_title}相关的摘要问题"),
            ),
        ),
    ]

    # 构建智能体
    agent = ReActAgent.from_tools(
        query_engine_tools,
        llm=llm,
        verbose=True,
    )

    agents[wiki_title] = agent

#### 为这些智能体定义IndexNode

In [ ]:
from llama_index.core.schema import IndexNode

# 定义顶级节点
objects = []
for wiki_title in wiki_titles:
    # 定义链接到这些智能体的索引节点
    wiki_summary = (
        f"此内容包含关于{wiki_title}的维基百科文章。如果"
        "您需要查找有关"
        f" {wiki_title}的具体事实，请使用此索引。\n如果您想要分析"
        "多个城市，请不要使用此索引。"
    )
    node = IndexNode(text=wiki_summary, index_id=wiki_title, obj=agents[wiki_title])
    objects.append(node)

#### 定义顶级检索器来选择智能体

In [ ]:
vector_index = VectorStoreIndex(
    objects=objects,
)
query_engine = vector_index.as_query_engine(similarity_top_k=1, verbose=True)

#### 测试查询

应该根据查询内容为特定智能体选择向量工具/摘要工具。

In [ ]:
# 应该使用Toronto智能体 -> 向量工具
response = query_engine.query("多伦多的人口是多少？")

In [12]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

In [ ]:
# 应该使用Houston智能体 -> 向量工具
response = query_engine.query("休斯顿是谁创建的以及何时创建的？")

In [14]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

In [ ]:
# 应该使用Boston智能体 -> 摘要工具
response = query_engine.query("总结波士顿的体育队伍")

In [16]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

In [ ]:
# 应该使用Seattle智能体 -> 摘要工具
response = query_engine.query("给我总结一下芝加哥的所有积极方面")

In [18]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))